Aquí utilizo todos los datos de los informes trimestrales para entrenar a un modelo que sea capaz de realizar predicciones trimestre a trimestre. De esta manera, y junto con el resto de predicciones, en aquellos casos en que todos apunten en la misma dirección, podría sentir cierta seguridad de que las predicciones van "en el buen camino".

# Tercer Modelo. Predicciones trimestrales

Comienzo como siempre importando los módulo que voy a utilizar.

In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from joblib import dump

import keras
from keras.engine import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, concatenate
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
set_random_seed(12)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

# Preprocessing

Aquí los chunks tendrán un tamaño de cuatro (4 trimestres = 1 año). Más me parecía excesivo y además acabaría con muchas menos entradas con las que poder entrenar al modelo.

In [2]:
# Sequence for the LSTM network
backlook = 4

# Size of data split for testing
train_size = 0.9

Cargo los datos de todas aquellas empresas que tengan al menos 5 informes trimestrales, elimino las columnas referentes a la moneda para quedarme únicamente con los datos numéricos, y junto los datos en bloques de 4 ordenados por fecha.

Al mismo tiempo, me creo otras dos listas, una de ellas tendrá los datos sobre el trimestre cuya variable objetivo tenemos que predecir (nexus_li), y la otra tendrá los valores que el modelo tiene que aprender a predecir (y_vals).

El objetivo es tener algo así (perdón por usar el horrible paint pero no quería complicarme de más):
![alt text](./quarterly-model-illustration.png "Esquema")

In [3]:
path = './datasets/fundamental_data'
all_files = glob.glob(path + "/*.csv")

time_li = []
nexus_li = []
y_vals = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0).sort_values(by='fiscalDateEnding')
    
    if len(df) > 4:
        df = df.drop(columns=['Unnamed: 0', 'fiscalDateEnding', 'reportedCurrency_x', 'reportedCurrency_y', 'reportedCurrency'])
        df = df.replace('None', 0).fillna(0)
        df = df[df['nextClossingVal'] != 0]
        
        for i in range(len(df)-backlook):
            time_li.append(df.iloc[i:i+backlook])
            y_vals.append(df.iloc[[i+backlook]]['nextClossingVal'].values[0])
            nexus_li.append(df.drop(columns='nextClossingVal').iloc[[i+backlook]])

Paso los datos a numpy para empezar a trabajar con ellos

In [4]:
time_arr = np.zeros((len(time_li), 4, 100))
nexus_arr = np.zeros((len(time_li), 99))
y_arr = np.array(y_vals).reshape(-1, 1)

for i in range(len(time_li)):
    time_arr[i] = time_li[i].to_numpy()
    nexus_arr[i] = nexus_li[i].to_numpy().reshape(99,)

Separo en train y test

In [5]:
# Train-Test split
split = int(len(time_arr) * train_size)

train_time = time_arr[:split]
train_nexus = nexus_arr[:split]
train_y = y_arr[:split]

test_time = time_arr[split:]
test_nexus = nexus_arr[split:]
test_y = y_arr[split:]

Normalizo y me guardo los normalizadores para cuando despliegue el modelo en google cloud

In [6]:
# Normalise series
time_normaliser = preprocessing.MinMaxScaler()
train_time = time_normaliser.fit_transform(train_time.reshape(train_time.shape[0]*train_time.shape[1], train_time.shape[2])).reshape(train_time.shape)
test_time = time_normaliser.transform(test_time.reshape(test_time.shape[0]*test_time.shape[1], test_time.shape[2])).reshape(test_time.shape)

# Normalise nexus
nexus_normaliser = preprocessing.MinMaxScaler()
train_nexus = nexus_normaliser.fit_transform(train_nexus)
test_nexus = nexus_normaliser.transform(test_nexus)

# Y normaliser
y_normaliser = preprocessing.MinMaxScaler()
train_y = y_normaliser.fit_transform(train_y)
test_y = y_normaliser.transform(test_y)

In [7]:
# Save scalers for future use
dump(time_normaliser, './normalisers/fundamental_x-time_normaliser.joblib')
dump(nexus_normaliser, './normalisers/fundamental_x-nexus_normaliser.joblib')
dump(y_normaliser, './normalisers/fundamental_y_normaliser.joblib')

['./normalisers/fundamental_y_normaliser.joblib']

# Model selection

##### Model constructor

In [8]:
def build_complex(x_time, x_nexus, y, batch_size=512, epochs=24, **params):
    
    # List of parameters
    if 'lstmsize' not in params: params['lstmsize'] = x_time.shape[1]
    if 'density' not in params: params['density'] = x_nexus.shape[1]
    if 'merge_density' not in params: params['merge_density'] = int((params['lstmsize']//1.5)*2) + x_nexus.shape[1]
    if 'activation' not in params: params['activation'] = 'relu'
    if 'twice' not in params: params['twice'] = False
    if 'optimizer' not in params: params['optimizer'] = 'adam'
    if 'shuffle' not in params: params['shuffle'] = False
        
    # Time model definition
    model_time = Sequential()
    
    model_time.add(LSTM(params['lstmsize'], input_shape=x_time.shape[1:], return_sequences=params['twice']))
    
    if 'dropout' in params:
        model_time.add(Dropout(params['dropout']))
    
    if params['twice']:
        model_time.add(LSTM(params['lstmsize']))
        
        if 'dropout' in params:
            model_time.add(Dropout(params['dropout']))
    
    # Nexus model definition
    model_nexus = Sequential()
    
    model_nexus.add(Dense(params['density'], input_shape=(99,), activation=params['activation']))
    
    # Ending pipe
    model = concatenate([model_time.output, model_nexus.output])
    
    model = Dense(params['merge_density'], activation=params['activation'])(model)
    
    if 'full_density' in params and params['full_density']:
        density = params['merge_density']//2
        while density >= 12:
            model = Dense(density, activation=params['activation'])(model)
            density //= 2
            
    model = Dense(1, activation='linear')(model)
    
    model = Model([model_time.input, model_nexus.input], model)
    
    model.compile(loss='mse', optimizer=params['optimizer'])
    
    if 'callbacks' in params:
        callback = model.fit(x=[x_time, x_nexus], y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'],
                            callbacks=params['callbacks'])
    else:
        callback = model.fit(x=[x_time, x_nexus], y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'])
    
    return [model, callback, params]

##### Evolutive algorith to search for the most optimal model

In [9]:
def set_genes(x_time, x_nexus, y, population_size, population=[]):
    if population_size-len(population) < 0:
        print('population size must be greater or equal than actual population')
        return
    
    if len(population) > 0:
        if len(population) < 3:
            print('population should be empty or 3 at least')
            return
        
        population[2] = breed_genes(x_time, x_nexus, y, population[0][2].copy())
        population[1] = breed_genes(x_time, x_nexus, y, combine=(population[0][2], population[1][2]))
        population = population[:3]
    
    for _ in range(population_size-len(population)):
        subject = breed_genes(x_time, x_nexus, y)
        population.append(subject)
        
    return population

def breed_genes(x_time, x_nexus, y, genes={}, combine=None):
    genes['x_time'] = x_time
    genes['x_nexus'] = x_nexus
    genes['y'] = y
    
    if type(combine) is list or type(combine) is tuple:
        genes['lstmsize'] = combine[np.random.randint(0,2)]['lstmsize']
        genes['density'] = combine[np.random.randint(0,2)]['density']
        genes['merge_density'] = combine[np.random.randint(0,2)]['merge_density']
        genes['activation'] = combine[np.random.randint(0,2)]['activation']
        genes['twice'] = combine[np.random.randint(0,2)]['twice']
        genes['optimizer'] = combine[np.random.randint(0,2)]['optimizer']
        genes['shuffle'] = combine[np.random.randint(0,2)]['shuffle']
        
        genes['dropout'] = combine[np.random.randint(0,2)].get('dropout')
        if genes['dropout'] is None: del genes['dropout']
        genes['full_density'] = combine[np.random.randint(0,2)].get('full_density')
        if genes['full_density'] is None: del genes['full_density']
            
    else:
        if np.random.randint(0,3) == 1:
            genes['lstmsize'] = int((np.random.randint(x_time.shape[1],x_time.shape[1]*2)//2)*2)

        if np.random.randint(0,3) == 1:
            genes['density'] = int((np.random.randint(x_nexus.shape[1]//2,x_nexus.shape[1]*2.66)//2)*2)

        if np.random.randint(0,3) == 1:
            genes['merge_density'] = int((np.random.randint(x_time.shape[1]+(x_nexus.shape[1]//2),
                                                            (x_time.shape[1]*2)+(x_nexus.shape[1]*2.66))//2)*2)

        if np.random.randint(0,3) == 1:
            activation = [
                'relu',
                'sigmoid',
                'softplus',
                'softsign',
                'tanh',
                'selu',
                'elu',
                'exponential'
            ]
            genes['activation'] = activation[np.random.randint(0,8)]

        if np.random.randint(0,3) == 1:
            genes['twice'] = True

        if np.random.randint(0,3) == 1:
            optimizer = [
                'sgd',
                'rmsprop',
                'adam',
                'adadelta',
                'adagrad',
                'adamax',
                'nadam'
            ]
            genes['optimizer'] = optimizer[np.random.randint(0,7)]

        if np.random.randint(0,3) == 1:
            genes['shuffle'] = True

        if np.random.randint(0,3) == 1:
            genes['dropout'] = np.random.randint(1,4)/10

        if np.random.randint(0,3) == 1:
            genes['full_density'] = True
            
    new_model = build_complex(**genes)
        
    return new_model

In [10]:
def breed_population(x_time, x_nexus, y, generations, population_size, population=[]):
    
    if generations > 1 and population_size < 3:
        print('population size should be of a minimum of 3 for more than one generation')
        return
    
    for g in range(generations):
        print(f'\nGENERATION {g}\n')
        population = set_genes(x_time, x_nexus, y, population_size, population)
        population = sorted(population, key=lambda x: x[1].history['val_loss'][-1]+x[1].history['loss'][-1])
        
    return population

In [11]:
best_models = breed_population(train_time, train_nexus, train_y, generations=20, population_size=10)


GENERATION 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [==============================] - 2s 87us/step - loss: 0.0075 - val_loss: 3.5815e-04
Epoch 2/24
18465/18465 [==============================] - 0s 22us/step - loss: 0.0012 - val_loss: 3.7740e-04
Epoch 3/24
18465/18465 [==============================] - 0s 23us/step - loss: 8.2777e-04 - val_loss: 2.3965e-04
Epoch 4/24
18465/18465 [==============================] - 0s 22us/step - loss: 6.8039e-04 - val_loss: 2.0667e-04
Epoch 5/24
18465/18465 [==============================] - 0s 22us/step - loss: 5.4245e-04 - val_loss: 1.7795e-04
Epoch 6/24
18465/18465 [==============================] - 0s 22us/step - loss: 4.2409e-04 - val_loss: 1.9066e-04
Epoch 7/24
18465/18465 [==============================] - 0s 22us/step - loss: 3.4177e-04 - val_loss: 1.3676e-04
Epoch 8/24
18465/18465 [=========

18465/18465 [==============================] - 0s 23us/step - loss: 9.8893e-05 - val_loss: 4.5542e-05
Epoch 18/24
18465/18465 [==============================] - 0s 22us/step - loss: 8.0580e-05 - val_loss: 3.3365e-05
Epoch 19/24
18465/18465 [==============================] - 0s 22us/step - loss: 7.6283e-05 - val_loss: 5.4734e-04
Epoch 20/24
18465/18465 [==============================] - 0s 22us/step - loss: 1.9315e-04 - val_loss: 6.8155e-05
Epoch 21/24
18465/18465 [==============================] - 0s 23us/step - loss: 7.3876e-05 - val_loss: 4.3689e-05
Epoch 22/24
18465/18465 [==============================] - 0s 23us/step - loss: 5.9922e-05 - val_loss: 9.4711e-05
Epoch 23/24
18465/18465 [==============================] - 0s 23us/step - loss: 7.3578e-05 - val_loss: 4.0637e-05
Epoch 24/24
18465/18465 [==============================] - 0s 23us/step - loss: 5.8480e-05 - val_loss: 3.2987e-05
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [===========================

Epoch 16/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.2571e-05 - val_loss: 6.9900e-05
Epoch 17/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.8881e-05 - val_loss: 6.2199e-05
Epoch 18/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.9216e-05 - val_loss: 7.2009e-05
Epoch 19/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.4594e-05 - val_loss: 4.2612e-05
Epoch 20/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.2374e-05 - val_loss: 4.6589e-05
Epoch 21/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.1682e-05 - val_loss: 5.3300e-05
Epoch 22/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.2057e-05 - val_loss: 4.5275e-05
Epoch 23/24
18465/18465 [==============================] - 0s 23us/step - loss: 4.7344e-05 - val_loss: 4.9538e-05
Epoch 24/24
18465/18465 [==============================] - 0s 24us/step - loss: 3.9113e-

Epoch 15/24
18465/18465 [==============================] - 0s 25us/step - loss: 5.4429e-05 - val_loss: 1.0662e-04
Epoch 16/24
18465/18465 [==============================] - 0s 25us/step - loss: 4.9394e-05 - val_loss: 8.4327e-05
Epoch 17/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.9825e-05 - val_loss: 7.6608e-05
Epoch 18/24
18465/18465 [==============================] - 0s 25us/step - loss: 5.2550e-05 - val_loss: 8.6592e-05
Epoch 19/24
18465/18465 [==============================] - 0s 25us/step - loss: 4.4806e-05 - val_loss: 7.9409e-05
Epoch 20/24
18465/18465 [==============================] - 0s 25us/step - loss: 4.3504e-05 - val_loss: 6.6676e-05
Epoch 21/24
18465/18465 [==============================] - 0s 25us/step - loss: 4.7899e-05 - val_loss: 7.8393e-05
Epoch 22/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.4217e-05 - val_loss: 7.9423e-05
Epoch 23/24
18465/18465 [==============================] - 0s 24us/step - loss: 4.2109e-

Epoch 14/24
18465/18465 [==============================] - 0s 26us/step - loss: 4.5929e-05 - val_loss: 7.6559e-05
Epoch 15/24
18465/18465 [==============================] - 0s 26us/step - loss: 5.5916e-05 - val_loss: 7.1159e-05
Epoch 16/24
18465/18465 [==============================] - 0s 26us/step - loss: 5.1444e-05 - val_loss: 2.6741e-05
Epoch 17/24
18465/18465 [==============================] - 0s 26us/step - loss: 4.5017e-05 - val_loss: 4.1769e-05
Epoch 18/24
18465/18465 [==============================] - 0s 26us/step - loss: 7.0434e-05 - val_loss: 4.7441e-05
Epoch 19/24
18465/18465 [==============================] - 0s 26us/step - loss: 7.1734e-05 - val_loss: 8.7314e-05
Epoch 20/24
18465/18465 [==============================] - 0s 26us/step - loss: 6.4847e-05 - val_loss: 8.2753e-05
Epoch 21/24
18465/18465 [==============================] - 0s 25us/step - loss: 7.4040e-05 - val_loss: 2.3281e-05
Epoch 22/24
18465/18465 [==============================] - 0s 26us/step - loss: 6.6000e-

...

Epoch 22/24
18465/18465 [==============================] - 1s 67us/step - loss: 0.0018 - val_loss: 5.8941e-05
Epoch 23/24
18465/18465 [==============================] - 1s 64us/step - loss: 3.3961e-04 - val_loss: 4.6080e-04
Epoch 24/24
18465/18465 [==============================] - 1s 64us/step - loss: 8.6628e-04 - val_loss: 3.0571e-04
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [==============================] - 37s 2ms/step - loss: 0.2422 - val_loss: 1.5641e-04
Epoch 2/24
18465/18465 [==============================] - 1s 66us/step - loss: 0.0010 - val_loss: 7.4062e-04
Epoch 3/24
18465/18465 [==============================] - 1s 65us/step - loss: 0.0022 - val_loss: 0.0014
Epoch 4/24
18465/18465 [==============================] - 1s 65us/step - loss: 0.0020 - val_loss: 7.3490e-04
Epoch 5/24
18465/18465 [==============================] - 1s 67us/step - loss: 0.0019 - val_loss: 7.8474e-04
Epoch 6/24
18465/18465 [==============================] - 1s 67us/step -

18465/18465 [==============================] - 1s 72us/step - loss: 3.4445e-04 - val_loss: 4.8635e-04
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [==============================] - 40s 2ms/step - loss: 0.0014 - val_loss: 1.2094e-04
Epoch 2/24
18465/18465 [==============================] - 1s 67us/step - loss: 5.4116e-04 - val_loss: 5.0626e-05
Epoch 3/24
18465/18465 [==============================] - 1s 70us/step - loss: 2.6095e-04 - val_loss: 4.5585e-05
Epoch 4/24
18465/18465 [==============================] - 1s 67us/step - loss: 2.5077e-04 - val_loss: 4.4731e-05
Epoch 5/24
18465/18465 [==============================] - 1s 67us/step - loss: 8.5078e-05 - val_loss: 4.8357e-05
Epoch 6/24
18465/18465 [==============================] - 1s 67us/step - loss: 6.1851e-05 - val_loss: 3.0146e-05
Epoch 7/24
18465/18465 [==============================] - 1s 67us/step - loss: 5.8076e-05 - val_loss: 2.8218e-05
Epoch 8/24
18465/18465 [==============================] - 1s 6

Epoch 23/24
18465/18465 [==============================] - 1s 69us/step - loss: 8.0058e-05 - val_loss: 6.4528e-05
Epoch 24/24
18465/18465 [==============================] - 1s 69us/step - loss: 7.7583e-05 - val_loss: 7.0443e-05
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [==============================] - 41s 2ms/step - loss: 0.2374 - val_loss: 7.8212e-04
Epoch 2/24
18465/18465 [==============================] - 1s 70us/step - loss: 0.0040 - val_loss: 4.8179e-04
Epoch 3/24
18465/18465 [==============================] - 1s 69us/step - loss: 9.8918e-04 - val_loss: 2.4146e-04
Epoch 4/24
18465/18465 [==============================] - 1s 69us/step - loss: 7.5843e-04 - val_loss: 2.1892e-04
Epoch 5/24
18465/18465 [==============================] - 1s 69us/step - loss: 6.3271e-04 - val_loss: 1.9581e-04
Epoch 6/24
18465/18465 [==============================] - 1s 69us/step - loss: 5.2839e-04 - val_loss: 1.6514e-04
Epoch 7/24
18465/18465 [=============================

18465/18465 [==============================] - 1s 70us/step - loss: 3.8206e-05 - val_loss: 6.7493e-05
Epoch 22/24
18465/18465 [==============================] - 1s 71us/step - loss: 5.9473e-05 - val_loss: 3.9100e-05
Epoch 23/24
18465/18465 [==============================] - 1s 73us/step - loss: 4.7373e-05 - val_loss: 5.1617e-05
Epoch 24/24
18465/18465 [==============================] - 1s 72us/step - loss: 5.0732e-05 - val_loss: 3.3943e-05
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [==============================] - 43s 2ms/step - loss: 0.0348 - val_loss: 7.8813e-04
Epoch 2/24
18465/18465 [==============================] - 1s 70us/step - loss: 0.0012 - val_loss: 1.5745e-04
Epoch 3/24
18465/18465 [==============================] - 1s 70us/step - loss: 9.5795e-04 - val_loss: 1.5854e-04
Epoch 4/24
18465/18465 [==============================] - 1s 69us/step - loss: 9.1680e-04 - val_loss: 1.4611e-04
Epoch 5/24
18465/18465 [==============================] - 1s 70

Epoch 20/24
18465/18465 [==============================] - 1s 73us/step - loss: 4.3469e-05 - val_loss: 3.5283e-05
Epoch 21/24
18465/18465 [==============================] - 1s 73us/step - loss: 4.3762e-05 - val_loss: 2.0942e-05
Epoch 22/24
18465/18465 [==============================] - 1s 72us/step - loss: 3.9845e-05 - val_loss: 1.7601e-05
Epoch 23/24
18465/18465 [==============================] - 1s 73us/step - loss: 3.7915e-05 - val_loss: 1.7503e-05
Epoch 24/24
18465/18465 [==============================] - 1s 72us/step - loss: 3.7852e-05 - val_loss: 1.9455e-05
Train on 18465 samples, validate on 2052 samples
Epoch 1/24
18465/18465 [==============================] - 47s 3ms/step - loss: 0.0838 - val_loss: 2.9259e-04
Epoch 2/24
18465/18465 [==============================] - 1s 73us/step - loss: 0.0015 - val_loss: 2.6462e-04
Epoch 3/24
18465/18465 [==============================] - 1s 72us/step - loss: 8.4360e-04 - val_loss: 1.2457e-04
Epoch 4/24
18465/18465 [==========================

In [12]:
best_models[0][1].history['val_loss']

[0.00013294491533552647,
 9.433451294093384e-05,
 0.00024443345026841084,
 5.084194772439731e-05,
 6.086511855305501e-05,
 2.8466620017275668e-05,
 0.00022664676140925755,
 9.240883775940437e-05,
 4.8228700760548635e-05,
 4.472073499405721e-05,
 2.4547553969038885e-05,
 2.7429191690933625e-05,
 4.1564037334845634e-05,
 2.310892973343327e-05,
 2.5157512207754807e-05,
 3.250769267976216e-05,
 2.8720995912282522e-05,
 6.419182370264096e-05,
 2.3574085557093465e-05,
 9.071044565133522e-05,
 7.243006553768214e-05,
 3.207610189724309e-05,
 2.511422016643025e-05,
 1.8176598550816367e-05]

In [13]:
print('optimal parameters')
print('_______________________')
for key,val in best_models[0][2].items():
    if key != 'x' and key != 'y': print(f'{key}: {val}')

optimal parameters
_______________________
twice: True
dropout: 0.2
shuffle: True
density: 54
activation: relu
lstmsize: 4
merge_density: 176
optimizer: adam
full_density: True


In [14]:
best_models[0][0].summary()

Model: "model_57"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_113_input (InputLayer)     (None, 4, 100)       0                                            
__________________________________________________________________________________________________
lstm_113 (LSTM)                 (None, 4, 4)         1680        lstm_113_input[0][0]             
__________________________________________________________________________________________________
dropout_113 (Dropout)           (None, 4, 4)         0           lstm_113[0][0]                   
__________________________________________________________________________________________________
lstm_114 (LSTM)                 (None, 4)            144         dropout_113[0][0]                
___________________________________________________________________________________________

In [15]:
checkpoint = ModelCheckpoint(
    filepath='./checkpoints/fundamental.{epoch}-{val_loss}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [16]:
params = best_models[0][2]
params['x_time'] = train_time
params['x_nexus'] = train_nexus
params['y'] = train_y
params['epochs'] = 2000
params['callbacks'] = [checkpoint]

In [17]:
final_model = build_complex(**params)

Train on 18465 samples, validate on 2052 samples
Epoch 1/2000
18465/18465 [==============================] - 49s 3ms/step - loss: 0.0031 - val_loss: 1.4996e-04
Epoch 2/2000
18465/18465 [==============================] - 1s 71us/step - loss: 7.5622e-04 - val_loss: 1.0301e-04
Epoch 3/2000
18465/18465 [==============================] - 1s 72us/step - loss: 5.6354e-04 - val_loss: 1.0758e-04
Epoch 4/2000
18465/18465 [==============================] - 1s 72us/step - loss: 3.3730e-04 - val_loss: 6.6341e-05
Epoch 5/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.7616e-04 - val_loss: 7.8136e-05
Epoch 6/2000
18465/18465 [==============================] - 1s 74us/step - loss: 1.0736e-04 - val_loss: 2.7518e-05
Epoch 7/2000
18465/18465 [==============================] - 1s 72us/step - loss: 5.7798e-05 - val_loss: 2.4425e-05
Epoch 8/2000
18465/18465 [==============================] - 1s 71us/step - loss: 5.6417e-05 - val_loss: 4.1140e-05
Epoch 9/2000
18465/18465 [=========

18465/18465 [==============================] - 1s 70us/step - loss: 2.9625e-05 - val_loss: 1.6435e-05
Epoch 142/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.6255e-05 - val_loss: 2.1109e-05
Epoch 143/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.9248e-05 - val_loss: 1.9141e-05
Epoch 144/2000
18465/18465 [==============================] - 1s 71us/step - loss: 3.1315e-05 - val_loss: 4.8921e-05
Epoch 145/2000
18465/18465 [==============================] - 1s 71us/step - loss: 3.7786e-05 - val_loss: 2.2848e-05
Epoch 146/2000
18465/18465 [==============================] - 1s 71us/step - loss: 3.0388e-05 - val_loss: 3.1628e-05
Epoch 147/2000
18465/18465 [==============================] - 1s 72us/step - loss: 3.9153e-05 - val_loss: 3.4131e-05
Epoch 148/2000
18465/18465 [==============================] - 1s 72us/step - loss: 3.8616e-05 - val_loss: 2.4914e-05
Epoch 149/2000
18465/18465 [==============================] - 1s 72us/step - lo

18465/18465 [==============================] - 1s 72us/step - loss: 2.7827e-05 - val_loss: 3.4081e-05
Epoch 282/2000
18465/18465 [==============================] - 1s 72us/step - loss: 3.1505e-05 - val_loss: 6.1061e-05
Epoch 283/2000
18465/18465 [==============================] - 1s 74us/step - loss: 3.2551e-05 - val_loss: 4.6534e-05
Epoch 284/2000
18465/18465 [==============================] - 1s 74us/step - loss: 2.7644e-05 - val_loss: 5.5332e-05
Epoch 285/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.6866e-05 - val_loss: 6.7460e-05
Epoch 286/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.9472e-05 - val_loss: 4.0139e-05
Epoch 287/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.8574e-05 - val_loss: 3.9439e-05
Epoch 288/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.5387e-05 - val_loss: 4.9307e-05
Epoch 289/2000
18465/18465 [==============================] - 1s 72us/step - lo

Epoch 351/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.5737e-05 - val_loss: 4.4189e-05
Epoch 352/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.6988e-05 - val_loss: 6.2751e-05
Epoch 353/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.9814e-05 - val_loss: 6.0914e-05
Epoch 354/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.4722e-05 - val_loss: 5.1313e-05
Epoch 355/2000
18465/18465 [==============================] - 1s 74us/step - loss: 2.9416e-05 - val_loss: 4.4629e-05
Epoch 356/2000
18465/18465 [==============================] - 1s 73us/step - loss: 2.7475e-05 - val_loss: 4.6546e-05
Epoch 357/2000
18465/18465 [==============================] - 1s 72us/step - loss: 3.0341e-05 - val_loss: 6.4045e-05
Epoch 358/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.9679e-05 - val_loss: 5.7683e-05
Epoch 359/2000
18465/18465 [==============================] - 1s

18465/18465 [==============================] - 1s 72us/step - loss: 2.3617e-05 - val_loss: 4.8090e-05
Epoch 422/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.3723e-05 - val_loss: 4.6618e-05
Epoch 423/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.7489e-05 - val_loss: 5.1066e-05
Epoch 424/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.4116e-05 - val_loss: 6.4987e-05
Epoch 425/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.5123e-05 - val_loss: 5.1734e-05
Epoch 426/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.4483e-05 - val_loss: 4.7707e-05
Epoch 427/2000
18465/18465 [==============================] - 1s 75us/step - loss: 2.2547e-05 - val_loss: 4.5280e-05
Epoch 428/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.4846e-05 - val_loss: 4.6000e-05
Epoch 429/2000
18465/18465 [==============================] - 1s 72us/step - lo

...

18465/18465 [==============================] - 1s 71us/step - loss: 2.0317e-05 - val_loss: 3.0033e-05
Epoch 1396/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.4936e-05 - val_loss: 3.1485e-05
Epoch 1397/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.0124e-05 - val_loss: 3.0899e-05
Epoch 1398/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.1739e-05 - val_loss: 3.0650e-05
Epoch 1399/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9585e-05 - val_loss: 2.5773e-05
Epoch 1400/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.0109e-05 - val_loss: 2.9843e-05
Epoch 1401/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9446e-05 - val_loss: 2.8553e-05
Epoch 1402/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.2745e-05 - val_loss: 2.9218e-05
Epoch 1403/2000
18465/18465 [==============================] - 1s 72us/s

18465/18465 [==============================] - 1s 72us/step - loss: 2.0615e-05 - val_loss: 2.5111e-05
Epoch 1534/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9700e-05 - val_loss: 2.2332e-05
Epoch 1535/2000
18465/18465 [==============================] - ETA: 0s - loss: 1.9504e-0 - 1s 75us/step - loss: 1.9415e-05 - val_loss: 2.3794e-05
Epoch 1536/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9696e-05 - val_loss: 2.1526e-05
Epoch 1537/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.0710e-05 - val_loss: 2.2731e-05
Epoch 1538/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9787e-05 - val_loss: 2.6605e-05
Epoch 1539/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9047e-05 - val_loss: 2.5860e-05
Epoch 1540/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9481e-05 - val_loss: 2.3985e-05
Epoch 1541/2000
18465/18465 [===============

18465/18465 [==============================] - 1s 72us/step - loss: 2.3380e-05 - val_loss: 1.7018e-05
Epoch 1672/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.0629e-05 - val_loss: 4.6922e-05
Epoch 1673/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9177e-05 - val_loss: 4.1627e-05
Epoch 1674/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9162e-05 - val_loss: 4.4552e-05
Epoch 1675/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.0511e-05 - val_loss: 1.6282e-05
Epoch 1676/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.1077e-05 - val_loss: 3.5031e-05
Epoch 1677/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.0828e-05 - val_loss: 4.1142e-05
Epoch 1678/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.0177e-05 - val_loss: 4.6255e-05
Epoch 1679/2000
18465/18465 [==============================] - 1s 75us/s

18465/18465 [==============================] - 1s 72us/step - loss: 2.1671e-05 - val_loss: 3.8829e-05
Epoch 1810/2000
18465/18465 [==============================] - 1s 75us/step - loss: 2.0234e-05 - val_loss: 5.9399e-05
Epoch 1811/2000
18465/18465 [==============================] - 1s 73us/step - loss: 2.8383e-05 - val_loss: 4.0034e-05
Epoch 1812/2000
18465/18465 [==============================] - 1s 71us/step - loss: 2.1540e-05 - val_loss: 3.9690e-05
Epoch 1813/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.1075e-05 - val_loss: 5.0189e-05
Epoch 1814/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.3945e-05 - val_loss: 3.7359e-05
Epoch 1815/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9635e-05 - val_loss: 4.0122e-05
Epoch 1816/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.1762e-05 - val_loss: 3.7974e-05
Epoch 1817/2000
18465/18465 [==============================] - 1s 72us/s

18465/18465 [==============================] - 1s 72us/step - loss: 2.0218e-05 - val_loss: 3.3959e-05
Epoch 1948/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.8680e-05 - val_loss: 4.0683e-05
Epoch 1949/2000
18465/18465 [==============================] - 1s 74us/step - loss: 2.3095e-05 - val_loss: 4.1067e-05
Epoch 1950/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9935e-05 - val_loss: 2.6423e-05
Epoch 1951/2000
18465/18465 [==============================] - 1s 72us/step - loss: 2.0702e-05 - val_loss: 3.3024e-05
Epoch 1952/2000
18465/18465 [==============================] - 1s 72us/step - loss: 1.9502e-05 - val_loss: 4.0578e-05
Epoch 1953/2000
18465/18465 [==============================] - 1s 73us/step - loss: 1.9377e-05 - val_loss: 2.9731e-05
Epoch 1954/2000
18465/18465 [==============================] - 1s 74us/step - loss: 2.0125e-05 - val_loss: 4.8174e-05
Epoch 1955/2000
18465/18465 [==============================] - 1s 71us/s

In [18]:
final_model[2]

{'twice': True,
 'dropout': 0.2,
 'shuffle': True,
 'density': 54,
 'activation': 'relu',
 'lstmsize': 4,
 'merge_density': 176,
 'optimizer': 'adam',
 'full_density': True,
 'callbacks': [<keras.callbacks.callbacks.ModelCheckpoint at 0x221c0bba208>]}

In [19]:
final_model[0].summary()

Model: "model_182"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_363_input (InputLayer)     (None, 4, 100)       0                                            
__________________________________________________________________________________________________
lstm_363 (LSTM)                 (None, 4, 4)         1680        lstm_363_input[0][0]             
__________________________________________________________________________________________________
dropout_363 (Dropout)           (None, 4, 4)         0           lstm_363[0][0]                   
__________________________________________________________________________________________________
lstm_364 (LSTM)                 (None, 4)            144         dropout_363[0][0]                
__________________________________________________________________________________________

In [51]:
final_model[0].load_weights('./checkpoints/fundamental.(best).hdf5')
y_predicted = final_model[0].predict([test_time, test_nexus])

In [52]:
predicted_y_test = y_normaliser.inverse_transform(y_predicted)
true_y_test = y_normaliser.inverse_transform(test_y)

In [53]:
loss = np.mean((true_y_test - predicted_y_test)**2)
medium_error = np.mean(abs(true_y_test - predicted_y_test))
print(f'MSE is {loss:.2f}\nMedium error is {medium_error:.2f}')

MSE is 119.15
Medium error is 5.81


In [54]:
up_down_test = []
up_down_predicted = []

for y,x in zip(test_y,test_nexus):
    if y[0] > x[-1]: up_down_test.append(1)
    elif y[0] < x[-1]: up_down_test.append(-1)
    else: up_down_test.append(0)
        
for y,x in zip(y_predicted,test_nexus):
    if y[0] > x[-1]: up_down_predicted.append(1)
    elif y[0] < x[-1]: up_down_predicted.append(-1)
    else: up_down_predicted.append(0)

In [55]:
print(f'Trend accuracy is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == p)/len(up_down_test))*100:.2f}%')

Trend accuracy is: 78.73%


In [56]:
print(f'Accuracy for upward trend is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == 1 and p == 1)/sum(1 for v in up_down_test if v == 1))*100:.2f}%')
print(f'Accuracy for downward trend is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == -1 and p == -1)/sum(1 for v in up_down_test if v == -1))*100:.2f}%')

Accuracy for upward trend is: 42.29%
Accuracy for downward trend is: 82.76%


In [60]:
print('Test growth trends:        Predicted growth trends:')
print(sum(1 for v in up_down_test if v > 0),'                      ',sum(1 for v in up_down_predicted if v > 0))
print('Test decreasing trends:    Predicted decreasing trends:')
print(sum(1 for v in up_down_test if v < 0),'                     ',sum(1 for v in up_down_predicted if v < 0))

Test growth trends:        Predicted growth trends:
227                        450
Test decreasing trends:    Predicted decreasing trends:
2053                       1830


In [58]:
print(f'Test mean: {np.mean(true_y_test)}')
print(f'Predictions mean: {np.mean(predicted_y_test)}')

Test mean: 32.93601798245614
Predictions mean: 31.7650089263916


##### Results seem pretty good!

In [59]:
print('Test values:    Predicted values:')
for r,p in zip(true_y_test.flatten(),predicted_y_test.flatten()):
    print(f'    {r:.2f}    {p:>12.2f}')

Test values:    Predicted values:
    40.89           30.40
    55.71           36.53
    51.07           50.62
    53.94           46.07
    51.45           48.67
    46.29           45.96
    38.68           41.07
    30.38           35.23
    17.39           27.31
    19.01           16.84
    30.15           27.08
    32.01           28.78
    35.67           29.59
    39.25           32.78
    40.64           36.31
    35.88           38.55
    32.84           33.05
    43.46           30.21
    32.91           40.42
    40.40           29.54
    45.11           36.58
    38.81           41.06
    41.69           35.10
    31.83           37.41
    43.19           29.49
    35.65           24.58
    31.79           31.21
    32.16           27.98
    33.12           28.26
    31.86           28.98
    31.16           27.97
    32.63           27.44
    33.65           28.70
    28.43           29.49
    33.63           25.39
    33.25           29.84
    34.11           29.54
    

    0.62            2.93
    5.15            8.01
    6.33            6.34
    7.18            7.27
    5.24            7.97
    4.97            6.44
    3.66            6.18
    2.72            5.24
    2.68            4.62
    1.05            4.60
    0.94            3.52
    0.38            3.45
    4.41            3.09
    1.26            5.78
    0.69            3.65
    0.56            3.29
    1.19            3.76
    1.46            3.64
    1.08            3.79
    1.91            3.54
    4.06            4.09
    4.37            5.54
    9.49            5.74
    17.42           10.02
    7.65           16.41
    9.94            8.25
    8.30           10.11
    6.88            8.77
    8.02            7.61
    4.68            8.57
    7.21            5.90
    29.90           27.99
    30.29           26.56
    26.39           26.87
    24.73           23.67
    23.99           22.32
    23.48           21.73
    27.20           21.29
    26.93           24.36
    24.64       

    9.98           11.78
    3.58           10.43
    4.04            5.39
    4.28            5.73
    0.89            6.04
    1.49            3.58
    1.39            4.04
    1.55            3.95
    1.64            4.08
    1.00            4.14
    1.74            3.73
    22.50           21.79
    22.19           20.69
    21.89           20.43
    14.21           20.20
    16.07           13.91
    18.67           15.33
    7.78           17.45
    13.00            8.36
    13.90           12.68
    11.87           13.36
    10.99           11.76
    6.17           11.05
    7.46            7.12
    5.77            6.56
    6.06            6.85
    7.60            7.07
    5.86            8.26
    7.19            7.00
    7.10            7.94
    7.15            7.91
    7.11            7.95
    6.99            7.92
    8.20            7.81
    9.91            8.76
    11.00           10.15
    11.95           11.06
    11.20           11.84
    15.59           11.26
    126.15 

    14.50           10.89
    35.05           13.87
    40.05           30.66
    40.05           35.35
    44.20           35.30
    49.60           39.23
    36.46           44.23
    55.01           31.71
    47.78           49.12
    40.04           42.43
    52.13           34.79
    24.73           46.42
    27.01           21.98
    30.00           25.82
    29.82           25.16
    28.20           25.61
    28.62           23.91
    28.29           24.08
    27.64           21.87
    30.28           26.30
    28.39           27.92
    28.70           25.95
    30.71           26.53
    26.02           26.51
    25.87           29.13
    25.73           22.68
    25.77           21.93
    25.36           22.54
    25.09           21.76
    25.09           21.40
    24.33           19.46
    25.69           23.87
    25.70           24.40
    26.00           23.88
    25.78           24.44
    22.80           22.81
    25.02           26.52
    24.50           21.86
    25.28   